<a href="https://colab.research.google.com/github/gizelleguerra/genAI_demos/blob/main/RAG_Llama2_Chroma_Colab_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on Tutorial from: https://www.youtube.com/watch?v=nac0nVBO24w

#**Install All the Required Packages**

In [1]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf

!pip -q install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip -q install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━

In [3]:
!pip -q install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.1 MB/s eta 0:00:00


In [54]:
!pip -q install gradio

NotImplementedError: ignored

#**Import All the Required Libraries**

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [5]:
from huggingface_hub import notebook_login
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

In [6]:
from langchain import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [7]:
import os
import sys

#**Load the Documents and Extract Text**

In [ ]:
# grab TIVDAK drug prescribing info from web
!mkdir -p DATA/TIVDAK;curl https://docs.seagen.com/Tivdak_Full_Ltr_Master.pdf -o DATA/TIVDAK/tivdak.pdf

In [ ]:
# save here
filepath = '/content/DATA/TIVDAK/tivdak.pdf'
loader = PyPDFLoader(filepath)
document=loader.load()

In [8]:
#!mkdir docs

In [12]:
# document=[]
# for file in os.listdir("docs"):
#   if file.endswith(".pdf"):
#     pdf_path="./docs/"+file
#     loader=PyPDFLoader(pdf_path)
#     document.extend(loader.load())
#   elif file.endswith('.docx') or file.endswith('.doc'):
#     doc_path="./docs/"+file
#     loader=Docx2txtLoader(doc_path)
#     document.extend(loader.load())
#   elif file.endswith('.txt'):
#     text_path="./docs/"+file
#     loader=TextLoader(text_path)
#     document.extend(loader.load())

In [14]:
len(document)

22

#**Split Document into Chunks**

In [15]:
document_splitter=CharacterTextSplitter(separator='\n', chunk_size=1000, chunk_overlap=100)

In [16]:
document_chunks=document_splitter.split_documents(document)

In [17]:
len(document_chunks)

70

In [18]:
document_chunks[0]

Document(page_content='HIGHLIGHTS OF PRESCRIBING INFORMATION  \nThese highlights do not include all the information needed to use \nTIVDAK  safely and effectively. See full prescribing information for \nTIVDAK . \nTIVDAK® (tisotumab vedotin -tftv) for injection, for intravenous use  \nInitial U.S. Approval: 2021  \n \nWARNING: OCULAR TOXICITY  \nSee full prescribing information for complete boxed warning.  \n \n• TIVDAK  caused changes in the corneal epithelium and \nconjunctiva resulting in changes in vision, including severe vision \nloss, and corneal ulceration . (5.1) \n• Conduct an ophthalmic exam at baseline , prior to each dose , and \nas clinically indicated . (2.2, 5.1) \n• Adhere to premedication and required eye care before, during, \nand after infusion.  (2.2) \n• Withhold TIVDAK  until improvement and resume, reduce the \ndose, or permanently discontinue, based on severity . (2.3, 5.1) \n ---------------------------  INDICATIONS AND USAGE  --------------------------', meta

In [19]:
document_chunks[1]

Document(page_content='---------------------------  INDICATIONS AND USAGE  --------------------------  \nTIVDAK is a tissue factor -directed antibody and microtubule inhibitor \nconjugate indicated for the treatment of adult patients with recurrent or \nmetastatic cervical cancer with disease progression on or after chemotherapy. \n(1) \nThis indication is approved under accelerated approval based on tumor \nresponse rate and durability of response. Continued approval for this \nindication may be contingent upon verification and description of clinical benefit in confirmatory trials. ( 14.1) \n ----------------------  DOSAGE AND ADMINISTRATION  ----------------------  \n• For intravenous infusion only.  Do not administer TIVDAK as an \nintravenous push or bolus. Do not mix with, or administer as an infusion \nwith,  other medicinal products. ( 2.4) \n• The recommended dose of TIVDAK is 2 mg/kg (up to a maximum of 200 \nmg) given as an intravenous infusion over 30 minutes every 3 weeks 

#**Download the Embeddings from Hugging Face**

In [20]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [21]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

#**Set Up Chroma as Vector Database**

Converting the Document Chunks into Embedding and save them to the vector store

In [22]:
vectordb=Chroma.from_documents(document_chunks,embedding=embeddings, persist_directory='./data')

In [23]:
vectordb.persist()

#**Login into Hugging Face Account to Download the Model**
use API key for your personal hugging face hub account

In [24]:
notebook_login()

#**Download the Llama 2 7B Chat Model**
download 4 bit version on google colab free tier

In [25]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              #load_in_8bit=True,
                                              load_in_4bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

#**Creating a Hugging Face Pipeline**

In [36]:
pipe=pipeline("text-generation",
              model=model,
              tokenizer=tokenizer,
              torch_dtype=torch.float16,
              device_map='auto',
              max_new_tokens=512,
              min_new_tokens=-1,
              top_k=30
              )

In [37]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [38]:
#llm=ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')

In [39]:
llm

HuggingFacePipeline(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7fbe7e646470>, model_id='gpt2', model_kwargs={'temperature': 0}, pipeline_kwargs=None)

#**Creating a memory object which is necessary to track inputs/outputs and hold a conversation**

In [40]:
memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)

#**Creating a Conversation Retrieval QA Chain**

The ConversationalRetrievalQA chain builds on RetrievalQAChain to provide a chat history component.



In [41]:
#Create our Q/A Chain
pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm,
                                             retriever=vectordb.as_retriever(search_kwargs={'k':6}),
                                             verbose=False, memory=memory)

In [32]:
#general question to test
#result=pdf_qa({"question":"YOLOv7 is trained on which dataset"})

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


In [33]:
#result['answer']

' YOLOv7 is trained on the COCO dataset.'

In [42]:
result=pdf_qa({"question":"what is Tivdak used for"})

In [43]:
result['answer']

' TIVDAK is a prescription medicine used to treat adults with cervical cancer that has returned or spread to other parts of the body, and who have received chemotherapy that did not work or is no longer working. It is not known if TIVDAK is safe and effective in children.'

In [44]:
result=pdf_qa({"question":"what are the risks in prescribing Tivdak"})

In [45]:
result['answer']

"  Tivdak is a tissue factor-directed antibody and microtubule inhibitor conjugate that is indicated for the treatment of adult patients with recurrent or metastatic cervical cancer with disease progression on or after chemotherapy. However, it is important to note that Tivdak has not been directly compared to rates in clinical trials of another drug, and its efficacy and safety have not been established in a large, randomized controlled trial. Additionally, Tivdak has been associated with various adverse reactions, including ocular toxicity, peripheral neuropathy, hemorrhage, pneumonitis, and embryo-fetal toxicity. Therefore, before prescribing Tivdak, clinicians should carefully evaluate the potential benefits and risks of this drug, taking into account the patient's individual circumstances and medical history."

# **Super simple chatbot interface**

In [52]:
print('---------------------------------------------------------------------------------')
print('Welcome to DrugGPT. You are now ready to start interacting with your drug prescribing guidelines')
print('---------------------------------------------------------------------------------')

while True:
  query=input(f"Prompt:")
  if query == "exit" or query == "quit" or query == "q" or query == "f":
    print('Exiting')
    sys.exit()
  if query == '':
    continue
  result = pdf_qa({"question": query})
  print(f"Answer: " + result["answer"])


---------------------------------------------------------------------------------
Welcome to the DrugGPT. You are now ready to start interacting with your drug prescribing guidelines
---------------------------------------------------------------------------------
Prompt:hello, who should Tivdak be prescribed to?
Answer:   Tivdak should be prescribed to adult patients with recurrent or metastatic cervical cancer with disease progression on or after chemotherapy. It is not recommended for use in pediatric patients or in patients with moderate or severe hepatic impairment. The recommended dose of Tivdak is 2 mg/kg given as an intravenous infusion over 30 minutes every 3 weeks until tumor progression. It is important to follow the dosage instructions provided by the healthcare provider and to not mix Tivdak with other medicinal products.
Prompt:what are the risks?
Answer:   Tivdak can cause serious side effects, including:

• Peripheral neuropathy. Nerve problems called peripheral neuropa

SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
